<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
import cv2
import ipywidgets.widgets as widgets
import threading
import time
import RPi.GPIO as GPIO

In [2]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [3]:
#初始化舵机
def initServo(port):
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False)
    GPIO.setup(port, GPIO.OUT)

In [4]:
#设置舵机角度
def setServoAngle(port,angle):
    tilt = GPIO.PWM(port, 50) #设置频率为50hz
    tilt.start(0)
    DutyCycle = angle/18 + 2
    tilt.ChangeDutyCycle(DutyCycle)
    time.sleep(1)
    tilt.stop()

In [5]:
#调整舵机
def adjVServo():
    initServo(11)
    initServo(23)
    status = input("Type 'h' to set horizontal angle, type 'v' to set vertical angle, type 'e' to end adjusting.")
    while (status == 'v') | (status=='h'):
        if status == 'v':
            port = 11
        else :
            port = 23
        angle = input('Please type an angle:')  # input输入的是字符串，需要用int（）函数转化成数字。
        angle = int(angle)
        setServoAngle(port,angle)
        status = input("'h' or 'v' or 'e' ?") 
    GPIO.cleanup()

In [6]:
#获取摄像头
def getCamera():
    #设置摄像头显示组件
    image_widget = widgets.Image(format='jpeg', width=500, height=400)  
    display(image_widget)   
    image = cv2.VideoCapture(0)                           #打开摄像头
    image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G')) #设置视频压制格式为MJPG
    
#     ##这部分参数取决于摄像头本身
#     image.set(3,600)  #设置帧宽
#     image.set(4,500)  #设置帧高
#     image.set(5,30)  #设置帧率
#     image.set(cv2.CAP_PROP_BRIGHTNESS, 55) #设置亮度 -64 - 64  0.0
#     image.set(cv2.CAP_PROP_CONTRAST, 55)   #设置对比度 -64 - 64  2.0
#     image.set(cv2.CAP_PROP_EXPOSURE, 140)  #设置曝光值 1.0 - 5000  156.0
    
    #读取摄像头数据
    ret, frame = image.read()     
    image_widget.value = bgr8_to_jpeg(frame)
    
    #将摄像头视频流赋值给显示组件
    while True:
        ret, frame = image.read()
        image_widget.value = bgr8_to_jpeg(frame)
        time.sleep(0.010)
    image.release()              #使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用

In [7]:
#分线程
threads = []

t1 = threading.Thread(target=getCamera)
threads.append(t1)
t2 = threading.Thread(target=adjVServo)
threads.append(t2)
for t in threads:
         t.start()
for t in threads:
         t.join()

Image(value=b'', format='jpeg', height='400', width='500')

Type 'h' to set horizontal angle, type 'v' to set vertical angle, type 'e' to end video session. h
Please type an angle: 90
'h' or 'v' or 'e' ? h
Please type an angle: 0
'h' or 'v' or 'e' ? h
Please type an angle: 180
'h' or 'v' or 'e' ? v
Please type an angle: 90
'h' or 'v' or 'e' ? v
Please type an angle: 120
'h' or 'v' or 'e' ? v
Please type an angle: 150
'h' or 'v' or 'e' ? v
Please type an angle: 120
'h' or 'v' or 'e' ? h
Please type an angle: 90
'h' or 'v' or 'e' ? v
Please type an angle: 120
'h' or 'v' or 'e' ? h
Please type an angle: 120
'h' or 'v' or 'e' ? h
Please type an angle: 75
'h' or 'v' or 'e' ? e


KeyboardInterrupt: 